In [8]:
import tensorflow as tf
import PIL
from tensorflow.python.framework import graph_io
from tensorflow.keras.models import load_model

In [4]:
tf.keras.backend.clear_session()

save_pb_dir = './model'
model_fname = '/home/jetson/classification/nets/weights-improvement-71-08-0.96.hdf5'
def freeze_graph(graph, session, output, save_pb_dir='.', save_pb_name='frozen_model.pb', save_pb_as_text=False):
    with graph.as_default():
        graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen = tf.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, save_pb_dir, save_pb_name, as_text=save_pb_as_text)
        return graphdef_frozen

# This line must be executed before loading Keras model.
tf.keras.backend.set_learning_phase(0) 

model = load_model(model_fname)

session = tf.keras.backend.get_session()

input_names = [t.op.name for t in model.inputs]
output_names = [t.op.name for t in model.outputs]

# Prints input and output nodes names, take notes of them.
print(input_names, output_names)

frozen_graph = freeze_graph(session.graph, session, [out.op.name for out in model.outputs], save_pb_dir=save_pb_dir)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
['input_1'] ['predictions/Softmax']
Instructions for updating:
Use `tf.compat.v1.graph_util.remove_training_nodes`
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 236 variables.
INFO:tensorflow:Converted 236 variables to const ops.


In [5]:
print(output_names, input_names)

['predictions/Softmax'] ['input_1']


In [6]:
import tensorflow.contrib.tensorrt as trt

trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,
    outputs=output_names,
    max_batch_size=1,
    max_workspace_size_bytes=1 << 25,
    precision_mode='FP16',
    minimum_segment_size=50
)

INFO:tensorflow:Linked TensorRT version: (5, 1, 6)
INFO:tensorflow:Loaded TensorRT version: (5, 1, 6)
INFO:tensorflow:Running against TensorRT version 5.1.6


In [7]:
graph_io.write_graph(trt_graph, "./model/",
                     "trt_graph.pb", as_text=False)

'./model/trt_graph.pb'